In [3]:
import cv2
import os
import glob 
from subprocess import check_output as call
import math
import numpy as np

def print_status(in_string):
    print('\n==> Running ', end =" ")
    
    for element in in_string:
        print(element, end =" ")
        
    return 0;

def extract_minutiae(tarp_loc, mindtct_loc, in_img, out_img, out_minutiae):
    img = cv2.imread(in_img,0)
    rows,cols = img.shape
    
    run_tarp = ["java","-jar", tarp_loc, in_img]
    #print_status(run_tarp)
    alignment = call(run_tarp).decode("utf-8").rstrip().split(" ")

    if len(alignment) != 3:
        # This means that the minutiae processing fails. The input image is terrible.
        return 1;

    alignment = [float(alignment[0]), float(alignment[1]), float(alignment[2])]
    angle = 360*(alignment[2]/(2*math.pi))

    M = np.float32([[1, 0, 150 - alignment[0]], [0, 1, 150 - alignment[1]]])
    dst = cv2.warpAffine(img, M, (cols, rows))

    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    dst2 = cv2.warpAffine(dst, M, (cols, rows))
    cv2.imwrite(out_img, dst2)
    
    run_mindtct = [mindtct_loc, out_img, out_minutiae]
    #print_status(run_mindtct)
    call(run_mindtct)

    return 0;


In [ ]:
files = glob.glob("/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/DB4_B/*.tif")
filetrim = "/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/DB4_B/"
alignedprefix = "/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/fingerprint_aligned/DB4_B/"
minutiaeprefix = "/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/minutiae/DB4_B/"
tarp_loc = '/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/main/resources/tarp.jar'
mindtct_loc = '/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/main/resources/mindtct'

In [ ]:
i = 0
print("total = %s" % len(files))

for f in files:
    fprefix = f.replace(".tif", "").replace(filetrim, "")
    aligned = alignedprefix + fprefix + ".jpeg"
    minutiae = minutiaeprefix + fprefix
    i = i + 1
    if i % 5 == 0:
        print(i)
    extract_minutiae(tarp_loc, mindtct_loc, f, aligned, minutiae)

In [11]:
files2 = glob.glob("/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/minutiae/DB2_B/*.xyt")
files3 = glob.glob("/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/minutiae/DB3_B/*.xyt")
files4 = glob.glob("/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/minutiae/DB4_B/*.xyt")
files = files2 + files3 + files4
print("total minutiae files = %s" % len(files))

x = []
y = []
o = []
for fname in files:
    contents = []
    with open(fname) as f:
        contents = f.readlines()
    for content in contents:
        content = content.rstrip().split(" ")
        x.append(int(content[0]))
        y.append(int(content[1]))
        o.append(int(content[2]))
        

total minutiae files = 235


In [19]:
print("x co-ordinate stats:")
print("cardinality of x: %s" % len(x))
print("mean: %s" %np.mean(x))
print("median: %s" %np.median(x))
print("std: %s" %np.std(x))
print("min: %s" %np.min(x))
print("max: %s" %np.max(x))

file = open("/Users/sashidharjakkamsetti/workspace/cs203/biohive/src/database/minutiae/probdist.meta", "w") 
file.write("%s %s %s %s %s\n" %(np.mean(x),np.median(x),np.std(x),np.min(x),np.max(x)))

x co-ordinate stats:
cardinality of x: 15070
mean: 150.5794293297943
median: 152.0
std: 70.37614760342565
min: 8
max: 295


48

In [20]:
print("y co-ordinate stats:")
print("cardinality of y: %s" % len(y))
print("mean: %s" %np.mean(y))
print("median: %s" %np.median(y))
print("std: %s" %np.std(y))
print("min: %s" %np.min(y))
print("max: %s" %np.max(y))

file.write("%s %s %s %s %s\n" %(np.mean(y),np.median(y),np.std(y),np.min(y),np.max(y)))

y co-ordinate stats:
cardinality of y: 15070
mean: 209.86735235567352
median: 212.0
std: 106.86636652590859
min: 7
max: 471


50

In [21]:
print("thetha stats:")
print("cardinality of thetha: %s" % len(o))
print("mean: %s" %np.mean(o))
print("median: %s" %np.median(o))
print("std: %s" %np.std(o))
print("min: %s" %np.min(o))
print("max: %s" %np.max(o))

file.write("%s %s %s %s %s\n" %(np.mean(o),np.median(o),np.std(o),np.min(o),np.max(o)))
file.close()

thetha stats:
cardinality of thetha: 15070
mean: 185.70942269409423
median: 202.0
std: 100.5168615375029
min: 0
max: 349
